Tento notebook připravuje agregovaná denní data výroby ze slunce a větru pro Česku a celou Evropu. Data se vypočítavají z koeficientů využití z datasetu PECD pro ERAA 2023 (viz níže). Hlavní analytický notebook je `spolehlivost-oze.ipynb`.


In [ ]:
library(nanoparquet)
library(readxl)
library(tidyverse)


# Načítání a transformace dat


## Instalované výkony

Předpokládané instalované výkony pro celoevropské analýzy na základě datasetu PEMMDB (*Pan-European Market Modelling Database*) pro celoevropské hodnocení zdrojové přiměřenosti ERAA (*European Resource Adequacy Assessment*) 2023.

Datasety jsou ke stažení [na stránkách ENTSO-E](https://www.entsoe.eu/eraa/2023/downloads/#Input%20Data). (Jedná se o datasety "PEMMDB Generation" pro instalované výkony a "PECD" pro hodinové profily výroby.)


In [ ]:
data_file_path <- function(file_name) {
  glue::glue("../data/pecd/{file_name}")
}

load_data_file <- function(name) {
  file_path <- data_file_path(name)
  if (xfun::file_ext(file_path) == "parquet") {
    return(read_parquet(file_path))
  }
  stop("Unknown data file format")
}

zone_to_country_code <- \(zone) substr(zone, 1, 2)

load_pemmdb_capacities <- \(target_year) {
  data_file_path("ERAA2023 PEMMDB Generation.xlsx") |>
  read_xlsx(
    sheet = str_glue("TY{target_year}"),
    range = "B2:BE25"
  ) |>
    rename(Source = 1) |>
    filter(Source %in% c("Solar (Photovoltaic)", "Wind Offshore", "Wind Onshore")) |>
    mutate(
      Source = case_match(
        Source,
        "Solar (Photovoltaic)" ~ "Solar",
        "Wind Offshore" ~ "Offshore",
        "Wind Onshore" ~ "Onshore"
      )
    ) |>
    pivot_longer(!Source, names_to = "Zone", values_to = "CapacityMW") |>
    filter(CapacityMW > 0) |>
    group_by(
      Source,
      Country = zone_to_country_code(Zone)
    ) |>
    summarise(CapacityMW = sum(CapacityMW), .groups = "drop")
}

# Projected installed capacities from ERAA data for use
# with the PECD time series.
df_installed <- load_pemmdb_capacities(target_year = 2025)


## Simulovaná výroba OZE z PECD

Zde načítáme simulovanou výrobu z větru a slunce z datasetu PECD (*Pan-European Climate Database*), přepočteme ji na denní a mezivýsledky uložíme. Napřed pro Česko, potom pro celou Evropu.


In [51]:
# Výroba ze slunce v Česku.
df_cz_daily_solar <- load_data_file("PECD-ERAA2023-LFSolarPV-2025.parquet") |>
  filter(country == "CZ") |>
  summarise(
    across(cf, mean),
    .by = year:day
  ) |>
  arrange(year, month, day)

df_cz_daily_solar |>
  write_parquet(data_file_path("eraa2023-ty2025-daily-solar-cz.parquet"))

# Výroba z větru v Česku.
df_cz_daily_wind <- load_data_file("PECD-ERAA2023-Wind_Onshore-2025.parquet") |>
  filter(country == "CZ") |>
  summarise(
    across(cf, mean),
    .by = year:day
  ) |>
  arrange(year, month, day)

df_cz_daily_wind |>
  write_parquet(data_file_path("eraa2023-ty2025-daily-wind-cz.parquet"))

# Kontrola, že roky počasí napříč datasety sedí.
years <- df_cz_daily_solar$year |> unique()
stopifnot(setequal(years, df_cz_daily_wind$year))


In [36]:
filter(df_installed, str_detect(Source, "(Off|On)shore")) |>
  pivot_wider(names_from = Source, names_prefix = "Capacity", values_from = CapacityMW)


Country,CapacityOffshore,CapacityOnshore
<chr>,<dbl>,<dbl>
BE,2260.0000,3598.00000
DE,11441.3000,69017.40000
DK,2845.6000,5318.27700
FR,2015.0000,24058.98870
IE,25.2000,5223.80000
IT,790.6984,14512.01978
NL,6139.0000,7300.04957
NO,3.5000,5423.22500
PT,88.0000,6399.50000


In [50]:
# Výroba z offshore větru v Evropě.
df_europe_daily_offshore <- load_data_file("PECD-ERAA2023-Wind_Offshore-2025.parquet") |>
  summarise(
    across(cf, mean),
    .by = c(country, year:day)
  )

# Výroba z onshore větru v Evropě.
df_europe_daily_onshore <- load_data_file("PECD-ERAA2023-Wind_Onshore-2025.parquet") |>
  summarise(
    across(cf, mean),
    .by = c(country, year:day)
  )

# Koeficienty využtí se váží předpokládanými instalovanými výkony
# v jednotlivých zemích z tabulky "PEMMDB Generation".

# Kombinovaná výroba z offshore a onshore větru v Evropě.
df_europe_daily_wind <- full_join(
    df_europe_daily_offshore,
    df_europe_daily_onshore,
    join_by(country, year, month, day),
    suffix = c("_off", "_on")
  ) |>
  right_join(
    filter(df_installed, str_detect(Source, "(Off|On)shore")) |>
      pivot_wider(
        names_from = Source,
        names_prefix = "Cap",
        values_from = CapacityMW
      ),
    join_by(country == Country)
  ) |>
  replace_na(
    list(
      CapOffshore = 0,
      CapOnshore = 0,
      cf_off = 0,
      cf_on = 0
    )
  ) |>
  summarise(
    cf = mean((cf_off * CapOffshore + cf_on * CapOnshore) / (CapOffshore + CapOnshore)),
    .by = year:day
  ) |>
  arrange(year, month, day)

df_europe_daily_wind |>
  write_parquet(data_file_path("eraa2023-ty2025-daily-wind-europe.parquet"))

# Výroba ze slunce (fotovoltaiky) v Evropě.
df_europe_daily_solar <- load_data_file("PECD-ERAA2023-LFSolarPV-2025.parquet") |>
  inner_join(
    filter(df_installed, Source == "Solar"),
    join_by(country == Country)
  ) |>
  summarise(
    cf = weighted.mean(cf, CapacityMW),
    .by = year:day
  ) |>
  arrange(year, month, day)

df_europe_daily_solar |>
  write_parquet(data_file_path("eraa2023-ty2025-daily-solar-europe.parquet"))

stopifnot(setequal(years, df_europe_daily_wind$year))
stopifnot(setequal(years, df_europe_daily_solar$year))
